In [1]:
pip install -Uqq fastbook

Note: you may need to restart the kernel to use updated packages.


In case "Add persistent_workers class variable to FakeLoader " error

please downgrade py torch version by using -> `conda install pytorch==1.6.0 torchvision==0.7.0 -c pytorch` command

* https://github.com/fastai/fastai/pull/2878
* https://github.com/lgvaz/faststyle/pull/18


In [4]:

#import tensorflow as tf
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc

from xgboost import XGBClassifier
import pickle5 as pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# load the baseline model
#learn.load('TubularLearner.fastAI._stage1')

#deployed_path = "TubularLearner.fastAI._stage2.pth"
# Load from file

XGBmodel = XGBClassifier()
XGBmodel.load_model("/mnt/vdb/thesis/model_v2/best_XGBmodel.model")

with open("/mnt/vdb/thesis/model_v2/best_RF.model", 'rb') as file:
    RF = pickle.load(file)

#DNN = load_model("../Models/best_DNN_model.hdf5")

#with open("../Models/best_SVM.model", 'rb') as file:
#    SVM = pickle.load(file)

In [6]:
# summarize model.
DNN.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1900)]            0         
_________________________________________________________________
dense (Dense)                (None, 896)               1703296   
_________________________________________________________________
dropout (Dropout)            (None, 896)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 448)               401856    
_________________________________________________________________
dropout_1 (Dropout)          (None, 448)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 224)               100576    
_________________________________________________________________
dropout_2 (Dropout)          (None, 224)              

In [5]:
df = pd.read_pickle("../datasets/truthset/AMPs_truthset.reps.plk")

In [6]:
predict_withModel(df,0, RF) # return class 1 (non AMP)

9

In [7]:
predict_withModel(df,0, XGBmodel) # return class 1 (non AMP)

6

# Predict on Shuffled sequences

In [4]:
df= pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
#df[df["length"] < 200] 
df

,ID,paper,seq,length,reps
0,ISGCock_Contig04_0915,PMC4864078,RQDRAHNQIDGQRPMGRLDKRKIAMKCTLSNQVFAKRRITYQTHKG...,50,"[0.01679776795208454, -0.07345905154943466, 0...."
1,ISGCock_Contig13_4610,PMC4864078,ICCLSTLMGNPRVKHLNKNNTYPKYIVLYLFHKLGFIQQLKHQC,44,"[0.015076708979904652, -0.050224486738443375, ..."
2,ISGCock_Contig16_2060,PMC4864078,VKHTKRTISYIYHKILRNSLMFTHINVHANGALK,34,"[0.020523976534605026, -0.03999472036957741, -..."
3,ISGCock_Contig16_4974,PMC4864078,KKIDKLIRPTVTLVHKSKFSESKAKILHTCQRFKKICPLLNRNW,44,"[0.02007628232240677, -0.05524342507123947, 0...."
4,ISGCock_Contig07_3736,PMC4864078,SKMCINSRYSTYSCCMRFKKHVYSFNNGFCQ,31,"[0.022961627691984177, -0.03958016261458397, -..."
...,...,...,...,...,...
57,TR42258|c1_g1_i1,hal-02965337,KSSVSLFFFPIVFFARLFVTALLRYRLFGLLMCSTLSPRWAFWV,44,"[0.014877736568450928, 0.00022438698215410113,..."
58,TR27534|c0_g1_i1,hal-02965337,CESFVGRRRVTIMRPFIREFKICRIRWGMSIYRRGKNVHELLRKLTCL,48,"[0.01740586943924427, -0.043963976204395294, 0..."
59,P3,S2162-2531(20)30132-3,SFSKKLRLWEPIWWPFRRLF,20,"[0.024964744225144386, -0.0190703384578228, -0..."
60,P10,S2162-2531(20)30132-3,LICWTVKLCVTLTHTTLTCTWNMR,24,"[0.02561492845416069, 0.008683688007295132, -0..."


In [13]:
# For test
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
predict_withModel(df_,0, RF)


[0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 0]


In [1]:
def predict_DNN(_df ,condition):
    if condition == 0:
        df = _df.copy()
    elif condition == -1 :
        df = _df.loc[_df.length <= 30 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy() # select records with lenght > 
    #print(df.columns)
    X= np.array(df['reps'].to_list())
    y_probas = DNN.predict(X)
    threshold = 0.5
    _y = np.where(y_probas > threshold, 1, 0)
    df['class'] = _y
    # accuracy = (len(df[df["class"] == 1])/len(df) )*100
    return len(df[df["class"] == 1])

def predict_withModel(_df ,condition, model):

    if condition == 0:
        df = _df.copy()
    elif condition == -1 :
        df = _df.loc[_df.length <= 30 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy() # select records with lenght > 
    #print(df.columns)
    X= np.array(df['reps'].to_list())
    _y = model.predict(X)
    #y_probas = learner.predict(X_test)
    #threshold = 0.5
    #_y = np.where(y_probas > threshold, 1, 0)
    
    df['class'] = _y

    #accuracy = (len(df[df["class"] == 1])/len(df) )*100

    return len(df[df["class"] == 1])

In [20]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## XGBoost

### XGBoost train with <=30

In [3]:

XGBmodel = XGBClassifier()
XGBmodel.load_model("../Models/best_XGBmodel._30.model")

In [4]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,21,4,7,12,12,18,3
shuff2,16,4,6,8,8,13,3
shuff3,16,4,7,12,12,16,0
shuff4,19,4,7,11,12,16,3
shuff5,19,4,7,9,9,14,5
shuff6,18,3,6,9,10,14,4
shuff7,15,2,5,7,7,10,5
shuff8,13,4,7,8,8,11,2
shuff9,17,4,7,9,11,15,2
shuff10,15,2,4,7,7,11,4


### XGBoost train with whole dataset|

In [8]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,XGBmodel)
result_df.loc[index, '>200']= predict_withModel(df_,200,XGBmodel)
result_df.loc[index, '>100']=predict_withModel(df_,100,XGBmodel)
result_df.loc[index, '>70']=predict_withModel(df_,70,XGBmodel)
result_df.loc[index, '>50']=predict_withModel(df_,50,XGBmodel)
result_df.loc[index, '>30']=predict_withModel(df_,30,XGBmodel)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,XGBmodel)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,19,4,6,12,13,15,4
shuff2,16,4,5,9,9,13,3
shuff3,22,3,5,11,13,17,5
shuff4,11,2,4,7,7,9,2
shuff5,16,4,4,6,7,11,5
shuff6,24,4,7,9,10,17,7
shuff7,15,3,5,6,10,12,3
shuff8,18,3,6,11,13,16,2
shuff9,18,3,5,11,13,16,2
shuff10,20,4,7,13,13,16,4


## SVM

In [6]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,29,4,7,16,19,25,4
shuff2,25,3,6,11,20,23,2
shuff3,23,3,3,8,11,18,5
shuff4,17,0,3,6,8,12,5
shuff5,26,1,4,9,14,22,4
shuff6,26,2,5,11,13,19,7
shuff7,21,1,4,10,11,14,7
shuff8,22,3,6,12,15,18,4
shuff9,19,0,0,2,10,14,5
shuff10,19,3,6,9,10,14,5


In [4]:

with open("/mnt/vdb/thesis/best_SVM.linear.model", 'rb') as file:
    SVM = pickle.load(file)

# SVM linear 
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,SVM)
result_df.loc[index, '>200']= predict_withModel(df_,200,SVM)
result_df.loc[index, '>100']=predict_withModel(df_,100,SVM)
result_df.loc[index, '>70']=predict_withModel(df_,70,SVM)
result_df.loc[index, '>50']=predict_withModel(df_,50,SVM)
result_df.loc[index, '>30']=predict_withModel(df_,30,SVM)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,SVM)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,18,4,7,12,13,16,2
shuff2,16,4,7,10,12,13,3
shuff3,18,4,6,13,14,17,1
shuff4,13,4,6,8,8,10,3
shuff5,20,4,7,9,11,17,3
shuff6,20,4,7,12,13,15,5
shuff7,14,4,7,10,10,12,2
shuff8,18,4,7,11,14,16,2
shuff9,20,4,5,9,15,17,3
shuff10,18,4,6,11,11,15,3


## RF

In [9]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_withModel(df_,0,RF)
result_df.loc[index, '>200']= predict_withModel(df_,200,RF)
result_df.loc[index, '>100']=predict_withModel(df_,100,RF)
result_df.loc[index, '>70']=predict_withModel(df_,70,RF)
result_df.loc[index, '>50']=predict_withModel(df_,50,RF)
result_df.loc[index, '>30']=predict_withModel(df_,30,RF)
result_df.loc[index, '30<=']=predict_withModel(df_,-1,RF)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,18,4,7,13,14,17,1
shuff2,12,4,6,8,8,12,0
shuff3,20,4,6,13,15,19,1
shuff4,13,4,6,10,10,13,0
shuff5,19,4,7,9,11,16,3
shuff6,16,4,6,10,12,14,2
shuff7,14,4,6,11,11,14,0
shuff8,13,4,7,9,9,12,1
shuff9,24,4,7,13,20,23,1
shuff10,17,4,6,11,12,16,1


## DNN

In [21]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,14,4,7,11,12,13,1
shuff2,12,4,6,10,10,12,0
shuff3,16,4,6,13,13,16,0
shuff4,13,4,7,8,8,10,3
shuff5,11,4,5,7,7,11,0
shuff6,19,4,7,11,13,16,3
shuff7,12,4,7,8,9,10,2
shuff8,12,4,7,7,9,11,1
shuff9,12,4,5,8,8,11,1
shuff10,14,4,7,9,10,12,2


In [31]:
result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,15,4,7,9,11,14,1
shuff2,20,4,7,11,15,19,1
shuff3,22,4,6,14,16,20,2
shuff4,14,4,7,7,7,11,3
shuff5,19,4,7,9,9,16,3
shuff6,19,4,7,13,13,17,2
shuff7,16,4,6,8,9,13,3
shuff8,16,4,7,10,11,15,1
shuff9,25,4,7,11,19,23,2
shuff10,19,4,6,11,11,16,3


Ten times

In [25]:
df = pd.read_pickle("../datasets/truthset/AMPs_truthset_10times.reps.plk")
predict(df)

62


predict AMP :48
predict NonAMP:14
predict NonAMP (in percentage):22.58064516129032


# PSSM

In [11]:
df = pd.read_pickle("../datasets/truthset/pssm/AMPs.truthset.reps.pkl")
df

,ID,paper,seq,reps
0,ISGCock_Contig04_0915,PMC4864078,ALQICTRNMIDDRLPYVADNVRPGTFIKQQRKQKQQRHHTSGTRKR...,"[0.5217362915633222, 0.4979000123479128, 0.497..."
1,ISGCock_Contig13_4610,PMC4864078,HLYPCKLNLKLGKVPFHFLNLNHKGKSIMVNQQTCLYYIICQTR,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
2,ISGCock_Contig16_2060,PMC4864078,ISHNHLTAASITHVKNRGKYIYMHLKFRKTNVLI,"[0.5213106112663236, 0.4979411881063055, 0.497..."
3,ISGCock_Contig16_4974,PMC4864078,RKKVWFIFHVCPKLKQRILSDTHAKNKCRLSPLLIKSTKIKNET,"[0.5082378908465902, 0.4992045461256411, 0.499..."
4,ISGCock_Contig07_3736,PMC4864078,CNYISFFRKCKNSQSTMYGCHRMNKCVFSSY,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
...,...,...,...,...
57,TR42258|c1_g1_i1,hal-02965337,MRYIVCFVFFLFFFLLFLWLVPARTASSFLTPRLSSLGKRSWAV,"[0.5246957976014336, 0.4976136544830712, 0.497..."
58,TR27534|c0_g1_i1,hal-02965337,MLFKVIIVIWISVCRECTRGGFCNFMHLKPISRELRRELYGRTRRRRK,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
59,P3,S2162-2531(20)30132-3,FWELWKFLKSLWSIFPRRRP,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
60,P10,S2162-2531(20)30132-3,ICTTLNWMVKLTCLTHVTLTTRWC,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."


## RF
19 - first

In [15]:
with open("/mnt/vdb/thesis/best_RF.model", 'rb') as file:
    RF = pickle.load(file)

In [16]:
predict_withModel(df,0, RF)

8

## XGBoost

In [17]:
XGBmodel = XGBClassifier()
XGBmodel.load_model("/mnt/vdb/thesis/best_XGBmodel.model")

In [18]:
predict_withModel(df,0, XGBmodel)

12

In [ ]:
SVM